In [23]:
# this is include zeno_etl_libs in the python search path on the run time
import sys
sys.path.append('./../..')

In [24]:
from zeno_etl_libs.helper import helper
from zeno_etl_libs.helper.aws.s3 import S3

In [25]:
import argparse
import io

import pandas as pd
import requests

from zeno_etl_libs.config import set_env_config, EnvNames
from zeno_etl_libs.db.db import DB, RedshiftSQLAlchemy

In [26]:
"""
Environment: Activate the env as per the requirement
"""
# env = EnvNames.production
# env = EnvNames.staging
env = EnvNames.development

In [27]:
config = set_env_config(env=env)

In [28]:
rs_db = DB(secrets=config.secrets)
rs_db.open_connection()

In [29]:
rs_db_sa = RedshiftSQLAlchemy(secrets=config.secrets)
rs_db_sa.open_connection()

In [30]:
s3 = S3(aws_access_key_id=config.secrets['AWS_ACCESS_KEY_ID'],
            aws_secret_access_key=config.secrets['AWS_SECRET_ACCESS_KEY_ID'])

In [9]:
report_type = 'installs_report'
ingore_columns = ["google-play-referrer"]

In [10]:
def get_data():
    appsflyer_uri = "https://hq.appsflyer.com/export"
    app_id = 'com.zenohealth.android'
    params = {
        'api_token': '70b710e8-59d1-4121-9aae-f520e4d0cebf',
        'from': '2022-02-01',
        'to': '2022-02-01',
        'timezone': 'Asia/Kolkata',
        'maximum_rows': 10000,
        'additional_fields': 'device_model,keyword_id,store_reinstall,deeplink_url,oaid,install_app_store,'
                             'contributor1_match_type,contributor2_match_type,contributor3_match_type,match_type,'
                             'device_category,gp_referrer,gp_click_time,gp_install_begin,amazon_aid,keyword_match_type,'
                             'att,conversion_type,campaign_type,is_lat'
    }

    url = '{}/{}/{}/v5'.format(appsflyer_uri, app_id, report_type)

    payload = {}

    res = requests.request("GET", url, data=payload, params=params)
    if res.status_code == 200:
        df = pd.read_csv(io.StringIO(res.text))
        # print(df)
        return df
    else:
        if res.status_code == 404:
            print('There is a problem with the request URL. Make sure that it is correct')
        else:
            print('There was a problem retrieving data: ', res.text)
    

In [11]:
def check_table_exists(dbcon, tablename):
    dbcur = dbcon.cursor()
    dbcur.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(tablename.replace('\'', '\'\'')))
    if dbcur.fetchone()[0] == 1:
        dbcur.close()
        return True

    dbcur.close()
    return False

In [12]:
def create_table_from_df(df, table_name, connection, schema="prod2-generico"):
    try:
        df.head(5).to_sql(
            name=table_name, 
            con=connection, 
            index=False, 
            if_exists='fail', 
            schema=schema)
        query = f"""truncate table "{schema}"."{table_name}"; """
        connection.execute(query)
        print(f"Created table: {table_name}, successfully.")
    except Exception as e:
        print(f"Error creating table: {e}")

In [13]:
df = get_data()
df.columns = [i.lower().replace(" ", "-") for i in df.columns]

In [14]:
table_name = "appsflyer-" + report_type.replace("_","-")

In [15]:
check_table_exists(dbcon=rs_db.connection, tablename=table_name)

True

In [33]:
table_info = helper.get_table_info(db=rs_db, table_name=table_name, schema='prod2-generico')

In [34]:
"""correcting the column order"""
df = df[table_info['column_name']]

In [16]:
create_table_from_df(df=df, table_name=table_name, connection=rs_db_sa.connection)

Error creating table: Table 'appsflyer-installs-report' already exists.


In [35]:
s3.write_df_to_db(df=df, table_name=table_name, db=rs_db, schema="prod2-generico")